In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
import pandas as pd

import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cathe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cathe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cathe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import numpy as np
import time
import datetime
import random
from nltk.corpus import stopwords
import re

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from nltk.corpus import wordnet

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [4]:
## to run it on Ca

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
# Transform to pd dataframe and print first rows
df = pd.read_csv("data\\split_data\\train_dataset.csv")
df.head(n=10)

,Unnamed: 0,text,target
0,0,A mother who repeatedly told health profession...,0
1,1,"According to her , she established Royal Seed ...",1
2,2,Political mercenaries were once again in hot d...,0
3,3,In a major step towards providing Universal He...,0
4,4,"Fortunately , the funding of the exhibition ha...",0
5,5,""" Data centers are especially vulnerable here ...",0
6,6,The number of children taking up smoking and w...,0
7,7,Every young person will one day have life-chan...,0
8,8,Vikran Patel CBS TV in an interview titled ' 6...,0
9,9,""" Then I tried taking the bus around town , it...",0


In [6]:
# This is the preprocessing from the tutorial. We may adapt it however we want.
# This one is VERY strict so it removes stopwords, emojis, punctuation, etc.

sw = stopwords.words('english')

def clean_text(text):

    text = text.lower()

    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text) # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")

    text = re.sub(r"http\S+", "",text) #Removing URLs
    #text = re.sub(r"http", "",text)

    html=re.compile(r'<.*?>')

    text = html.sub(r'',text) #Removing html tags

    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'') #Removing punctuations

    text = [word.lower() for word in text.split() if word.lower() not in sw]

    text = " ".join(text) #removing stopwords

    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) #Removing emojis

    return text


In [7]:
# Applied preprocessing


# for index, row in df.iterrows():
#     if type(row['text']) == float:
#         print(row['text'])
#         print(index)

df['text'] = df['text'].apply(lambda x: clean_text(x))
df.head()
len(df)

9422

In [8]:
sentences = df.text.values
labels = df.target.values
random_seed = 42 

In [9]:
def split_data_randomly(sentences, labels, train_size, random_seed=None):
    """
    Split the data randomly into training and validation sets.

    Parameters:
    - sentences: numpy array containing sentences
    - labels: numpy array containing corresponding labels
    - train_size: proportion of the dataset to include in the training set
    - random_seed: seed for random number generator (optional)

    Returns:
    - train_sentences: numpy array containing training sentences
    - train_labels: numpy array containing corresponding training labels
    - val_sentences: numpy array containing validation sentences
    - val_labels: numpy array containing corresponding validation labels
    """

    # Set random seed if provided
    if random_seed:
        np.random.seed(random_seed)

    # Shuffle the data
    shuffled_indices = np.random.permutation(len(sentences))
    shuffled_sentences = sentences[shuffled_indices]
    shuffled_labels = labels[shuffled_indices]

    # Calculate the number of samples for training
    train_samples = int(len(sentences) * train_size)

    # Split the data into training and validation sets
    train_sentences = shuffled_sentences[:train_samples]
    train_labels = shuffled_labels[:train_samples]
    val_sentences = shuffled_sentences[train_samples:]
    val_labels = shuffled_labels[train_samples:]

    return train_sentences, train_labels, val_sentences, val_labels

train_size = 0.8  # 80% of the data for training

train_sentences, train_labels, val_sentences, val_labels = split_data_randomly(sentences, labels, train_size, random_seed=42)

print(len(train_sentences))
print(len(val_sentences))
print(len(train_labels))
print(len(val_labels))

7537
1885
7537
1885


In [10]:
AUGMENTATION_SAMPLE_COUNT = 4
random.seed(42) 

def random_delete(sentence):
    """
    Pick a random index in a sentence, the word associated with that index is to be deleted.

    Arguments: 
        sentence: sentence string before random word removal
    Returns: 
        new_sentences: list of sentence strings after random word removal
    """
    words = sentence.split()
    new_sentences = []

    
    
    if len(words) > AUGMENTATION_SAMPLE_COUNT:
        
        # select one random word to remove for each new sample (3 new samples in total)
        random_indices =  np.random.choice(np.arange(0, len(words) + 1), size=AUGMENTATION_SAMPLE_COUNT , replace=False)

        # remove random word
        for random_index in random_indices:
            new_words = words.copy()
            try:
                new_words = words[:random_index] + words[random_index+1:]
                new_sentence = " ".join(new_words)

                # add to the list of new sentences based on the current sample
                new_sentences.append(new_sentence)
            except IndexError:
                pass
        
    return new_sentences



In [11]:
def get_synonyms(word):
    """
    Given a word, use WordNet to get a list of synonyms
    """
    synonyms = []
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            if word != l.name():
                synonyms.append(l.name())
    return synonyms



In [12]:
AUGMENTATION_SAMPLE_COUNT = 4

def synonym_replace(sentence):
    """
    Pick a random index in a sentence. Replace the word associated with that index with a random synonym. 
    (Assumes that stopwords, punctuation have already been removed)

    Arguments: 
        sentence: sentence string before augmentation
    Returns: 
        new_sentences: list of sentence strings after augmentation
    """
    words = sentence.split()
    new_sentences = []

    
    
    if len(words) > AUGMENTATION_SAMPLE_COUNT :
        # select one random word to remove for each new sample 
        random_indices =  np.random.choice(np.arange(0, len(words) + 1), size=AUGMENTATION_SAMPLE_COUNT, replace=False)

        # remove random word
        for random_index in random_indices:
            new_words = words.copy()
            
            try:
                synonym = random.choice(get_synonyms(words[random_index]))
                new_words[random_index] = synonym
                new_sentence = " ".join(new_words)

                print(new_words)

                # add to the list of new sentences based on the current sample
                new_sentences.append(new_sentence)
            except IndexError:
                pass
        
    return new_sentences



In [13]:
# List for all the new sentences created through random deletion
all_new_stcs = []

# Loop through sentences in training data
# If the sentence is a positive sample (minority class),
# create new sentences by randomly selecting one word to remove for each new sentence
for stc_idx, stc in enumerate(train_sentences):
    if labels[stc_idx] == 1:
        new_stcs_batch = synonym_replace(stc)

        # Add the new sentences created from this individual sample to all new sentences list
        all_new_stcs.extend(new_stcs_batch)

# convert to numpy array and concatenate to previous sentence array
array_all_new_stcs = np.array(all_new_stcs)
train_sentences = np.concatenate((train_sentences, array_all_new_stcs), axis=0)

# add n positive labels (n being the number of new sentences) to the labels array
array_all_new_labels = np.array([1] * len(all_new_stcs))
train_labels = np.concatenate((train_labels, array_all_new_labels), axis=0)

print(len(train_sentences))
print(len(val_sentences))
print(len(train_labels))
print(len(val_labels))

############ end of data augmentation ######################

['vatican', 'controversy', ',', 'mcdonald', 'helps', 'feed', 'homeless', 'Rome']
['vatican', 'controversy', ',', 'mcdonald', 'help', 'feed', 'homeless', 'rome']
['vatican', 'controversy', ',', 'mcdonald', 'helps', 'provender', 'homeless', 'rome']
['juliet', 'otami', ',', 'central', 'regional', 'human', 'resource', 'manager', 'Gold_Coast', 'education', 'service', 'ges', ',', 'said', 'recent', 'reports', 'sexual', 'abuses', 'male', 'teachers', 'must', 'wakeup', 'call', 'female', 'teachers', 'champion', 'need', 'protect', 'vulnerable', 'school', 'girls', 'falling', 'prey', 'male', 'teachers']
['juliet', 'otami', ',', 'central', 'regional', 'human', 'resource', 'manager', 'ghana', 'education', 'service', 'ges', ',', 'said', 'recent', 'reports', 'intimate', 'abuses', 'male', 'teachers', 'must', 'wakeup', 'call', 'female', 'teachers', 'champion', 'need', 'protect', 'vulnerable', 'school', 'girls', 'falling', 'prey', 'male', 'teachers']
['hong', 'kong', ',', 'coastal', 'city', ',', 'vulnerabl

In [14]:
# This just shows how it has been tokenized and id-mapped. We can delete.
print(' Original: ', train_sentences[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(train_sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(train_sentences[0])))

 Original:  still able walk , said jackson therapy , travel st ann bay per week proving costly using walker hobble around first , treatment working wonders , due financial difficulties discontinue therapy need financial assistance , unable work farm
Tokenized:  ['still', 'able', 'walk', ',', 'said', 'jackson', 'therapy', ',', 'travel', 'st', 'ann', 'bay', 'per', 'week', 'proving', 'costly', 'using', 'walker', 'ho', '##bble', 'around', 'first', ',', 'treatment', 'working', 'wonders', ',', 'due', 'financial', 'difficulties', 'disco', '##nti', '##nu', '##e', 'therapy', 'need', 'financial', 'assistance', ',', 'unable', 'work', 'farm']
Token IDs:  [2145, 2583, 3328, 1010, 2056, 4027, 7242, 1010, 3604, 2358, 5754, 3016, 2566, 2733, 13946, 17047, 2478, 5232, 7570, 11362, 2105, 2034, 1010, 3949, 2551, 16278, 1010, 2349, 3361, 8190, 12532, 16778, 11231, 2063, 7242, 2342, 3361, 5375, 1010, 4039, 2147, 3888]


In [15]:
# Training
# This actually tokenizes all the sentences and updates the max length so the model knows where to pad the sentences.
max_len = 0

# For every sentence...
for sent in train_sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    train_input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(train_input_ids))

print('Max sentence length: ', max_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (610 > 512). Running this sequence through the model will result in indexing errors


Max sentence length:  610


In [16]:
# Validation
# This actually tokenizes all the sentences and updates the max length so the model knows where to pad the sentences.
# max_len = 0

# For every sentence...
for sent in val_sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    val_input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(val_input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  610


In [17]:
train_input_ids = []
train_attention_masks = []

# For every stc...
for stc in train_sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    train_encoded_dict = tokenizer.encode_plus(
                        stc,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        truncation=True,
                        #max_length = max_len,           # Pad & truncate all sentences.
                        padding = 'max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    train_input_ids.append(train_encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    train_attention_masks.append(train_encoded_dict['attention_mask'])

# Convert the lists into tensors.
train_input_ids = torch.cat(train_input_ids, dim=0)
train_attention_masks = torch.cat(train_attention_masks, dim=0)
train_labels = torch.tensor(train_labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', train_sentences[0])
print('Token IDs:', train_input_ids[0])

Original:  still able walk , said jackson therapy , travel st ann bay per week proving costly using walker hobble around first , treatment working wonders , due financial difficulties discontinue therapy need financial assistance , unable work farm
Token IDs: tensor([  101,  2145,  2583,  3328,  1010,  2056,  4027,  7242,  1010,  3604,
         2358,  5754,  3016,  2566,  2733, 13946, 17047,  2478,  5232,  7570,
        11362,  2105,  2034,  1010,  3949,  2551, 16278,  1010,  2349,  3361,
         8190, 12532, 16778, 11231,  2063,  7242,  2342,  3361,  5375,  1010,
         4039,  2147,  3888,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,   

In [18]:
val_input_ids = []
val_attention_masks = []

# For every stc...
for stc in val_sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    val_encoded_dict = tokenizer.encode_plus(
                        stc,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        truncation=True,
                        #max_length = max_len,           # Pad & truncate all sentences.
                        padding = 'max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    val_input_ids.append(val_encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    val_attention_masks.append(val_encoded_dict['attention_mask'])

# Convert the lists into tensors.
val_input_ids = torch.cat(val_input_ids, dim=0)
val_attention_masks = torch.cat(val_attention_masks, dim=0)
val_labels = torch.tensor(val_labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', val_sentences[0])
print('Token IDs:', val_input_ids[0])

Original:  home generations immigrants belgians north african descent , molenbeek feels like dead end many youth , residents say opportunities , feel neglected authorities rejected rest belgian society giving opening radicals seek recruits
Token IDs: tensor([  101,  2188,  8213,  7489,  6995,  2015,  2167,  3060,  6934,  1010,
        16709, 27698,  4402,  2243,  5683,  2066,  2757,  2203,  2116,  3360,
         1010,  3901,  2360,  6695,  1010,  2514, 15486,  4614,  5837,  2717,
         6995,  2554,  3228,  3098, 23618,  6148, 15024,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     

In [19]:
# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)

# Combine the validation inputs into a TensorDataset.
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)


# Create a 90-10 train-validation split.

# # Calculate the number of samples to include in each set.
# train_size = int(0.8 * len(dataset))
# #val_size = int(0.2 * len(dataset))
# val_size = len(dataset)  - train_size

# # Divide the dataset by randomly selecting samples.
# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# print('{:>5,} training samples'.format(train_size))
# print('{:>5,} validation samples'.format(val_size))

print(len(train_dataset), ' training samples')
print(len(val_dataset), ' validation samples')

print(train_dataset[:10])

9710  training samples
1885  validation samples
(tensor([[  101,  2145,  2583,  ...,     0,     0,     0],
        [  101, 12111,  6704,  ...,     0,     0,     0],
        [  101,  2852, 24810,  ...,     0,     0,     0],
        ...,
        [  101,  2105,  2454,  ...,     0,     0,     0],
        [  101,  2279,  1010,  ...,     0,     0,     0],
        [  101,  1010,  7272,  ...,     0,     0,     0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0]))


In [20]:
# # Combine the training inputs into a TensorDataset.
# dataset = TensorDataset(input_ids, attention_masks, labels)

# # Create a 90-10 train-validation split.

# # Calculate the number of samples to include in each set.
# train_size = int(0.8 * len(dataset))
# #val_size = int(0.2 * len(dataset))
# val_size = len(dataset)  - train_size

# # Divide the dataset by randomly selecting samples.
# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# print('{:>5,} training samples'.format(train_size))
# print('{:>5,} validation samples'.format(val_size))

In [21]:
# The DataLoader needs to know our batch size for training, so we specify it
# here. For fine-tuning BERT on a specific task, the authors recommend a batch
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order.
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [22]:
# Load BertForSequenceClassification, the pretrained BERT model with a single
# linear classification layer on top.
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# enable gradient checkpointing to avoid out of memory with GPU training
model.gradient_checkpointing_enable()

# if device == "cuda:0":
# # Tell pytorch to run this model on the GPU.
#     model = model.cuda()
model = model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

C:\Users\cathe\anaconda3\envs\pcl-573\Lib\site-packages\transformers\optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [24]:
# Number of training epochs
epochs = 2

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [25]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [26]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [27]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the device using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        output = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels)
        loss = output.loss
        total_train_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables
    total_eval_accuracy = 0
    best_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():
            output= model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        loss = output.loss
        total_eval_loss += loss.item()
        # Move logits and labels to CPU if we are using GPU
        logits = output.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    if avg_val_accuracy > best_eval_accuracy:
        torch.save(model, 'bert_model')
        best_eval_accuracy = avg_val_accuracy
    #print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    #print("  Validation took: {:}".format(validation_time))
    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 2 ========
Training...

  Average training loss: 0.56
  Training epoch took: 0:07:35

Running Validation...
  Accuracy: 0.87

======== Epoch 2 / 2 ========
Training...

  Average training loss: 0.45
  Training epoch took: 0:07:31

Running Validation...
  Accuracy: 0.90

Training complete!
Total training took 0:15:52 (h:mm:ss)


In [28]:
model = torch.load('bert_model')

In [29]:
df_test = pd.read_csv('data\\split_data\\dev_dataset.csv')

# for i in df_test['text']:
#     try: clean_text(i) 
#     except AttributeError:
#         print(i)

df_test['text'] = df_test['text'].apply(lambda x:clean_text(x))
test_sentences = df_test['text'].values

In [30]:
test_input_ids = []
test_attention_masks = []
for stc in test_sentences:
    encoded_dict = tokenizer.encode_plus(
                        stc,
                        truncation=True,
                        padding = 'max_length',
                        add_special_tokens = True,
                        # max_length = max_len,
                        # pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    test_input_ids.append(encoded_dict['input_ids'])
    test_attention_masks.append(encoded_dict['attention_mask'])
test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)

In [31]:
test_dataset = TensorDataset(test_input_ids, test_attention_masks)
test_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [32]:
predictions = []
for batch in test_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        with torch.no_grad():
            output= model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask)
            logits = output.logits
            logits = logits.detach().cpu().numpy()
            pred_flat = np.argmax(logits, axis=1).flatten()

            predictions.extend(list(pred_flat))

In [33]:
df_output = pd.DataFrame()
#df_output['id'] = df_test['id'] # Do we need to add ids?
df_output['target'] =predictions
df_output.to_csv('D3_BERT_updatedRandomDeleteWithReplace_4samples',index=False, header=False)